## dataframe으로 저장된 opera trial balance를 이용해서, 메입카드사 별 금액과 건 수 정렬

In [22]:
import pandas as pnds
import pickle
import datetime

# dataframe으로 저장된 opera trial balance 읽기
yesterday = datetime.datetime.now() - datetime.timedelta(1) # 어제 날짜 추출
target_date = yesterday.strftime("%Y%m%d")                  # 어제 날짜 포맷
target_date = '20220710'

dfdata_dir = './dfdata/' + target_date + '/'
df_filename = 'df_kicc_history_' + target_date               # 파일 이름: df_kicc_history_YYYYMMDD
# 임시
df_filename = 'df_kicc_history_20220710'

try:
    with open(dfdata_dir + df_filename, "rb") as file:
        card_history_df = pickle.load(file)
except Exception as e:
    with open('./error.log', 'a') as file:
        file.write(
            f'[{__name__}.py] <{datetime.datetime.now()}> pickle file-reading error {dfdata_dir + df_filename} : {e}\n'
        )
        print(
            f'[{__name__}.py] <{datetime.datetime.now()}> pickle file-reading error {dfdata_dir + df_filename} : {e}'
        )
    raise(e)

card_history_df

,거래고유번호,승인구분,date,승인번호,카드번호,발급카드사,매입카드사,금액
49,100333809558,승인,2022-07-10,073285,5567-1743-2036-4885,해외마스타,하나구외환,75794
48,100322711257,승인,2022-07-10,035815,5567-1738-0771-4075,해외마스타,하나구외환,75794
47,100333809970,승인,2022-07-10,013650,5567-1733-1411-0965,해외마스타,하나구외환,75794
46,100312712551,승인,2022-07-10,067837,5567-1702-4609-8771,해외마스타,하나구외환,85364
45,100311713438,승인,2022-07-10,057926,5567-1742-3541-7869,해외마스타,하나구외환,85364
44,100311714027,승인,2022-07-10,078785,5567-1724-9976-5783,해외마스타,하나구외환,299153
43,100333810896,승인,2022-07-10,067719,5567-1734-4102-3719,해외마스타,하나구외환,121539
42,100311715714,승인,2022-07-10,31339554,4265-8692-1246-4648,KB 기업카드,국민카드,69000
41,100312716524,승인,2022-07-10,31339587,4265-8692-1246-4648,KB 기업카드,국민카드,69000
40,100333811665,승인,2022-07-10,31339622,4265-8692-1246-4648,KB 기업카드,국민카드,69000


In [23]:
grouped_df = card_history_df.groupby(['매입카드사']).aggregate({'승인번호': 'count', '금액': 'sum'})
print(grouped_df)


       승인번호       금액
매입카드사               
국민카드     14  1225501
비씨카드      8  2213320
삼성카드      4   198940
신한카드      1    92233
하나구외환    20  3422514
현대카드      3   130300


In [24]:
# 6. EDI 엑셀에 입력하기위한 카드 분류
# '카드분류' 컬럼 생성
card_history_df['카드분류'] = '분류'        # 'LT', 'KEB', 'JCB', 'VISA', 'MASTER', 'SA', 'SS', 'SH', 'BC', 'KB', 'HD', 'NH', 'CITI'

# 'LT'
condition = (card_history_df['매입카드사'].str.startswith('롯데', na=False))
collected_cards = card_history_df[condition].index.tolist()
card_history_df.loc[collected_cards, '카드분류'] = '9244'

# 'KEB'
condition = (card_history_df['매입카드사'] == '하나구외환')  & (card_history_df['발급카드사'].str.startswith('하나', na=False) | card_history_df['발급카드사'].str.startswith('토스', na=False))
collected_cards = card_history_df[condition].index.tolist()
card_history_df.loc[collected_cards, '카드분류'] = '9243'

# 'JCB'
condition = (card_history_df['발급카드사'].str.contains('제이씨비', na=False))
collected_cards = card_history_df[condition].index.tolist()
card_history_df.loc[collected_cards, '카드분류'] = '9235'

# 'VISA'
condition = (card_history_df['발급카드사'].str.contains('해외비자', na=False))
collected_cards = card_history_df[condition].index.tolist()
card_history_df.loc[collected_cards, '카드분류'] = '9231'

# 'MASTER'
condition = (card_history_df['발급카드사'].str.contains('해외마스타', na=False))
collected_cards = card_history_df[condition].index.tolist()
card_history_df.loc[collected_cards, '카드분류'] = '9233'

# 'SA'
condition = (card_history_df['발급카드사'].str.contains('해외아멕스', na=False))
collected_cards = card_history_df[condition].index.tolist()
card_history_df.loc[collected_cards, '카드분류'] = '9232'

# 'SS'
condition = (card_history_df['발급카드사'].str.startswith('삼성', na=False)) & (card_history_df['발급카드사'].str.contains('비씨', na=False) == False)
collected_cards = card_history_df[condition].index.tolist()
card_history_df.loc[collected_cards, '카드분류'] = '9239'

# 'SH'
condition = (card_history_df['매입카드사'].str.startswith('신한', na=False))
collected_cards = card_history_df[condition].index.tolist()
card_history_df.loc[collected_cards, '카드분류'] = '9242'

# 'BC'
condition = (card_history_df['매입카드사'].str.startswith('비씨', na=False))      # 'CITI'씨티카드와 중복되지만, 'CITI'는 마지막에 다시 셋팅
collected_cards = card_history_df[condition].index.tolist()
card_history_df.loc[collected_cards, '카드분류'] = '9237'

# 'KB'
condition = (card_history_df['매입카드사'].str.startswith('국민', na=False))
collected_cards = card_history_df[condition].index.tolist()
card_history_df.loc[collected_cards, '카드분류'] = '9238'

# 'HD'
condition = (card_history_df['매입카드사'].str.startswith('현대', na=False))
collected_cards = card_history_df[condition].index.tolist()
card_history_df.loc[collected_cards, '카드분류'] = '9245'

# 'NH'
condition = (card_history_df['매입카드사'].str.startswith('NH', na=False))
collected_cards = card_history_df[condition].index.tolist()
card_history_df.loc[collected_cards, '카드분류'] = '9240'

# 'CITI'
condition = (card_history_df['발급카드사'].str.startswith('씨티', na=False)) & (card_history_df['발급카드사'].str.contains('비씨', na=False) == False)
collected_cards = card_history_df[condition].index.tolist()
card_history_df.loc[collected_cards, '카드분류'] = '9241'

# 어제와 오늘 날짜의 파일이 섞여 있는 현 원본을 따로 저장


card_history_df.sort_values(['카드분류'])

,거래고유번호,승인구분,date,승인번호,카드번호,발급카드사,매입카드사,금액,카드분류
1,111321281332,승인,2022-07-11,161221,3752-8003-5764-005,해외아멕스,삼성카드,106590,9232
49,100333809558,승인,2022-07-10,073285,5567-1743-2036-4885,해외마스타,하나구외환,75794,9233
29,100633847088,승인,2022-07-10,557421,5137-9680-0171-1038,해외마스타,하나구외환,77000,9233
30,100612807491,승인,2022-07-10,536002,5137-9680-0171-1038,해외마스타,하나구외환,77000,9233
31,100633845366,승인,2022-07-10,28OVY5,5405-4250-0307-3870,해외마스타,하나구외환,79718,9233
19,101522844289,승인,2022-07-10,376714,5440-3641-0417-1332,해외마스타,하나구외환,126028,9233
11,110122949523,승인,2022-07-11,313019,5137-9680-0073-5038,해외마스타,하나구외환,142200,9233
9,110222959627,승인,2022-07-11,754495,5395-9388-3445-0465,해외마스타,하나구외환,58370,9233
10,110233432532,승인,2022-07-11,766600,5557-9776-2145-1967,해외마스타,하나구외환,50742,9233
2,111212726842,승인,2022-07-11,871075,5386-4600-0114-2545,해외마스타,하나구외환,145200,9233


In [25]:
# 그룹핑
grouped_df = card_history_df.groupby(['카드분류']).aggregate({'승인번호': 'count', '금액': 'sum'})
print(grouped_df)


      승인번호       금액
카드분류               
9232     1   106590
9233    17  1832914
9237     8  2213320
9238    14  1225501
9239     3    92350
9242     1    92233
9243     3  1589600
9245     3   130300


In [26]:
map_code_edi = {
    # TRX_CODE : 엑셀좌표
    '9231': 'C7',
    '9232': 'C9',
    '9233': 'C8',
    '9234': 'C15',
    '9235': 'C6',
    '9236': 'C3',
    '9237': 'C12',
    '9238': 'C14',
    '9239': 'C10',
    '9240': 'C16',
    '9241': 'C17',
    '9242': 'C11',
    '9243': 'C5',
    '9244': 'C4',
    '9245': 'C15'
}
    
# 카드 트랙잭션 코드별 금액 리스트 딕셔너리
actual_cards_dict = {
    # {'transaction code': [금액,],}
    '9231': ['='],
    '9232': ['='],
    '9233': ['='],
    '9234': ['='],
    '9235': ['='],
    '9237': ['='],
    '9238': ['='],
    '9239': ['='],
    '9240': ['='],
    '9241': ['='],
    '9242': ['='],
    '9243': ['='],
    '9244': ['='],
    '9245': ['=']
}

# 코드별로 해당되는 각 리스트에 금액을 추가, 나중에 코드별로 join해서 edi 엑셀 파일에 저장(map_code_edi 참조)
for index, row in card_history_df.iterrows():                   # dataframe row looping
    actual_cards_dict[row['카드분류']].append(f"+{row['금액']}")     # 금액을 str 타입으로 바꾸어서 해당 리스트에 저장


In [27]:
# import openpyxl
# import os

# # EDI 템플릿 엑셀 파일을 읽어들임
# excel_filename = './EDI-xx월.xlsx'                                      # EDI 엑셀 템플릿 파일

# try:
#     edi_excel = openpyxl.load_workbook(excel_filename, data_only=False) # 수식파일 포함하여 엑셀파일 읽어 들임
#     ws = edi_excel.active                                               # 활성화된 sheet
# except Exception as e:
#     with open('./error.log', 'a') as file:
#         file.write(
#             f'[{__name__}.py] <{datetime.datetime.now()}> openpyxl file-reading error {excel_filename} : {e}\n'
#         )
#         print(
#             f'[{__name__}.py] <{datetime.datetime.now()}> openpyxl file-reading error {excel_filename} : {e}\n'
#         )
#     raise(e)

# # 4. trial balance 내용을 순서대로 돌면서 엑셀에 입력
# for code, amounts in actual_cards_dict.items():     # trx-code를 이용해서 ws와 opera_df에 접근하는 looping
#     ws[map_code_edi[code]] = ''.join(amounts)
# edi_excel.save('./arrange.xlsx')